In [ ]:
#%cd /hdd1/kotensekiocrmodel/mmdetection/
!pip3 list | grep mm

In [ ]:
import mmengine
cfg = mmengine.Config.fromfile('/root/ndlkotenocr_cli/src/ndl_kotenseki_layout/models/ndl_kotenseki_layout_config_v3.py')

In [ ]:
cfg.model.backbone.arch="tiny"
cfg.checkpoint_file = "/root/ndlkotenocr_cli/src/ndl_kotenseki_layout/models/ndl_kotenseki_layout_ver3.pth"
cfg.model.neck.in_channels=[96, 192, 384, 768]
cfg.model.backbone.init_cfg.checkpoint=cfg.checkpoint_file
cfg.model.test_cfg.rcnn.max_per_img=1000

In [ ]:
## アノテーションフォルダの設定(jsonファイル等の作成方法はcococonverter-NDLDocL.ipynbを参照のこと。)
train_label_path = "/root/layoutinputs/kotensekicocotrain/kotensekicocotrain.json"
val_label_path = "/root/layoutinputs/kotensekicocovalid/kotensekicocovalid.json"

# 画像ファイルのパス
train_img_path = "/root/layoutinputs/kotensekicocotrain/img/"
val_img_path = "/root/layoutinputs/kotensekicocovalid/img/"

# ルートフォルダ
root_path = "/root/layoutinputs/layoutinputs/"
# 作業フォルダ（学習時のモデル出力先（エポック毎に出力))
work_path = '/root/layoutinputs/kotensekiocrmodel/work_dir_mmdetv3_cascademaskrcnn-convnextv1_{}'.format(cfg.model.backbone.arch)

In [ ]:

from mmdet.registry import DATASETS
from mmdet.datasets.coco import CocoDataset


@DATASETS.register_module()
class TinyDataset(CocoDataset):
    metainfo = {
        'classes': ('text', ),
        'palette': [
            (220, 20, 60),
        ]
    }

In [ ]:
# コンフィグを変更する

# 作業フォルダ
cfg.work_dir = work_path

# rootフォルダ
cfg.data_root = root_path
cfg.dataset_type='TinyDataset'
cfg.train_dataloader.dataset.type= cfg.dataset_type
cfg.val_dataloader.dataset.type= cfg.dataset_type
cfg.test_dataloader.dataset.type = cfg.dataset_type

# アノテーションファイルのパス
cfg.test_dataloader.dataset.data_root=root_path
cfg.test_dataloader.dataset.ann_file=val_label_path
cfg.test_dataloader.dataset.data_prefix.img=val_img_path
cfg.test_evaluator.ann_file=val_label_path

cfg.train_dataloader.dataset.data_root=root_path
cfg.train_dataloader.dataset.ann_file=train_label_path
cfg.train_dataloader.dataset.data_prefix.img=train_img_path

cfg.val_dataloader.dataset.data_root=root_path
cfg.val_dataloader.dataset.ann_file=val_label_path
cfg.val_dataloader.dataset.data_prefix.img=val_img_path
cfg.val_evaluator.ann_file=val_label_path

#cfg.data.samples_per_gpu=32
cfg.train_dataloader.batch_size=24
cfg.train_dataloader.num_workers=6
cfg.val_dataloader.batch_size=1
cfg.val_dataloader.num_workers=6
cfg.test_dataloader.batch_size=1
cfg.test_dataloader.num_workers=6
cfg.load_from=None
#学習済みモデルを利用する場合
#cfg.load_from="/root/ndlkotenocr_cli/src/ndl_kotenseki_layout/models/ndl_kotenseki_layout_ver3.pth"

cfg.train_pipeline = train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='RandomFlip', prob=0.5),
    dict(
        type='RandomChoice',
        transforms=[
            [
                dict(
                    type='RandomChoiceResize',
                    scales=[(400, 1333), (500, 1333), (600, 1333)],
                    keep_ratio=True),
                dict(
                    type='RandomCrop',
                    crop_type='absolute_range',
                    crop_size=(384, 600),
                    allow_negative_crop=True),
                dict(
                    type='RandomChoiceResize',
                    scales= [(480, 1333), (512, 1333), (544, 1333),
                                    (576, 1333), (608, 1333), (640, 1333),
                                    (672, 1333), (704, 1333), (736, 1333),
                                    (768, 1333), (800, 1333)],
                    keep_ratio=True)
            ],
        ]),
    dict(type='PackDetInputs')
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None, _scope_='mmdet'),
    dict(type='Resize', scale=(1333, 800), keep_ratio=True, _scope_='mmdet'),
    dict(
        type='LoadAnnotations',
        with_bbox=True,
        with_mask=True,
        _scope_='mmdet'),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'),
        _scope_='mmdet')
]

cfg.train_dataloader.dataset.pipeline=cfg.train_pipeline
cfg.test_dataloader.dataset.pipeline=cfg.test_pipeline

cfg.model.roi_head.bbox_head[0].num_classes=1
cfg.model.roi_head.bbox_head[1].num_classes=1
cfg.model.roi_head.bbox_head[2].num_classes=1
cfg.model.roi_head.mask_head.num_classes=1


# Set seed thus the results are more reproducible
cfg.seed = 0
cfg.gpu_ids = range(1)
cfg.device = 'cuda'
cfg.metainfo = {
    'classes': ('text', ),
    'palette': [
        (220, 20, 60),
    ]
}
#cfg.image_size = (2048, 2048)
#cfg.runner.meta = dict(fp16 = dict(init_scale=512))
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

# 編集したコンフィグをファイル出力する場合
#cfg.dump('/root/kotensekiocrmodel202312/mmdetv3-cascade_mask_rcnn_convnextv1-{}.py'.format(cfg.model.backbone.arch))

In [ ]:
#from mmdet.datasets import build_dataset
#from mmdet.models import build_detector
from mmengine.registry import RUNNERS
#from mmdet.apis import train_detector
import os.path as osp
import os
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP


os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'

# initialize the process group
dist.init_process_group("gloo", rank=0, world_size=1)


# Create work_dir
os.makedirs(osp.abspath(cfg.work_dir),exist_ok=True)
#train_detector(model, datasets, cfg, distributed=False, validate=True)
runner = RUNNERS.build(cfg)
runner.train()